### Master oficial en Big Data & Data Science - Universidad Internacional de Valencia (VIU)

<font color="darkblue">

-   **Alumno:** Franco Rojas Yucra
-   **TFM:** Sistema para la busqueda de conocimientos en textos de documentos multimedia
-   **Email:** franco.rojasyucra@alumnos.viu.es
-   **Fecha:** Octubre de 2023
<font color="black">

## 09 Implementacion de modelo de lenguaje con Langchain

Librerías utilizadas

In [1]:
import re
import string
import pandas as pd
from langchain import HuggingFacePipeline
from langchain import PromptTemplate
from langchain import LLMChain
import re
from dash import Dash, dcc, html, Input, Output, State, callback

In [2]:
pd.set_option('display.max_columns',None) #motrar todas las columnas

Lectura de modelo con HuggingFacePipeline

In [3]:
llm = HuggingFacePipeline.from_model_id(model_id="franco-rojas/bloom-1b1-finetuned-tfmviu",
                                        task="text-generation",
                                        model_kwargs={"temperature": 0.2, "max_length": 200})

C:\Users\Franco\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.0.1+cu118 with CUDA 1108 (you have 2.0.1+cpu)
    Python  3.9.13 (you have 3.9.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


Template que ayuda a generar pregunta y respuesta

In [4]:
template = """<|prompter|>{question}<|endoftext|><|assistant|>"""
prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [5]:
#Función para retirar codigos de template
def answer_str(question,txt_lg):
    txt1=re.sub(question, '', txt_lg)
    txt2=re.sub(f'<|assistant|>', '', txt1)
    txt3=re.sub(f'<|prompter|>', '', txt2)
    txt_resp=re.sub('[%s]' % re.escape('?¿|'), '', txt3)
    return txt_resp.strip()

Configuración Dash para interfaz

In [6]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = Dash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div([
    dcc.Input(id='input-1-state',style={'width': '500px'}, type='text', value=''),
    html.Button(id='submit-button-state', n_clicks=0, children='CONSULTAR'),
    html.Div(id='output-state')
])
@callback(Output('output-state', 'children'),
              Input('submit-button-state', 'n_clicks'),
              State('input-1-state', 'value'))
def update_output(n_clicks, input1):
    answ=llm_chain.run(input1)
    txt_resp=answer_str(input1,answ)
    return f'''
        Respuesta: "{str(txt_resp)}"
    '''

###### Interfaz de respuesta para modelo BLOOM

Escribe una consulta en el cuadro:

In [7]:
if __name__ == '__main__':
    app.run(debug=True)

C:\Users\Franco\anaconda3\lib\site-packages\transformers\generation\utils.py:1270: UserWarning:

You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )

C:\Users\Franco\anaconda3\lib\site-packages\transformers\generation\utils.py:1369: UserWarning:

Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.

